# 0. Imports

### 0.1 Libs

In [137]:
import re
import warnings
import inflection
import sqlite3
import pickle


import numpy            as np
import pandas           as pd
import seaborn          as sns
import umap.umap_       as umap
import matplotlib.cm    as cm
import scipy.stats      as st

from IPython.display        import Image
from sqlalchemy             import create_engine
from pandas_profiling       import ProfileReport

from plotly                 import express          as px
from matplotlib             import pyplot           as plt

from scipy                  import stats            as ss
from scipy.cluster          import hierarchy        as hc

from sklearn                import cluster          as c
from sklearn                import metrics          as m
from sklearn                import preprocessing    as pp
from sklearn                import decomposition    as dd
from sklearn                import ensemble         as en
from sklearn                import mixture          as mx
from sklearn                import manifold         as mn

### 0.2 Helper Functions

In [138]:
warnings.simplefilter("ignore")

def numerical_metrics(numerical_attributes):
    #ct = central tendency
    ct_mean = pd.DataFrame(numerical_attributes.apply(np.mean)).T
    ct_median = pd.DataFrame(numerical_attributes.apply(np.median)).T

    #d = dispersion
    d_std = pd.DataFrame(numerical_attributes.apply(np.std)).T
    d_min = pd.DataFrame(numerical_attributes.apply(min)).T
    d_max = pd.DataFrame(numerical_attributes.apply(max)).T
    d_range = pd.DataFrame(numerical_attributes.apply(lambda x: x.max() - x.min())).T
    d_skew = pd.DataFrame(numerical_attributes.apply(lambda x: x.skew())).T
    d_kurtosis = pd.DataFrame(numerical_attributes.apply(lambda x: x.kurtosis())).T

    concat = pd.concat([d_min,d_max,d_range,ct_mean,ct_median,d_std,d_skew,d_kurtosis]).T.reset_index()
    concat.columns=['Attributes','Min','Max','Range','Mean','Median','STD','Skew','Kurtosis']

    return concat

### 0.3 Loading Data

In [139]:
# df_raw = pd.read_csv('../data/data.csv', encoding= 'unicode_escape')

path_s3 = 's3://vhmf-insiders-dataset/'
df_raw = pd.read_csv(path_s3 + 'data.csv', encoding= 'unicode_escape')

# 1. Data Description

### 1.0 Rename Columns

In [86]:
df1 = df_raw.copy()

df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [87]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore(x)

cols_new = list( map(snakecase, cols_old))

df1.columns = cols_new

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### 1.1 Data Dimentions

In [88]:
print(f'Number of Rows:{df1.shape[0]}')
print(f'Number of Columns:{df1.shape[1]}')
print('Number of Clients:{}'.format(len(df1['customer_id'].unique())))

Number of Rows:541909
Number of Columns:8
Number of Clients:4373


### 1.4 Fillout NA

In [89]:
df1 = df1.dropna(subset=['description', 'customer_id'])
print('Removed data: {:.0%}'.format( 1-(df1.shape[0] / df_raw.shape[0])))
print(f'New number of rows: {df1.shape[0]}')

Removed data: 25%
New number of rows: 406829


### 1.5 Change Types

In [90]:
df1['customer_id'] = df1['customer_id'].astype(int)

df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])
df1['invoice_date'] = df1['invoice_date'].dt.strftime('%Y-%m-%d')
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom


# 2. Data Filtering 

### 2.1 Bad Users

In [91]:
df2 = df1.copy()

In [92]:
df2 = df2[~df2['customer_id'].isin([16446])]  # Maior número de devolução do conjunto de dados
df2 = df2[~df2['customer_id'].isin([17548, 16546, 15823, 13672, 12346, 13762, 18268, 14557, 16878, 13364, 14792, 12607, 12454, 18274, 12558])]  # itens devolvidos >= itens comprados



### 2.2 Cat Attributes

In [93]:
# Stock Code != ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']
# df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK'])]

# Description
df2 = df2.drop(columns = 'description', axis = 1)

### 2.3 Num Attributes

In [94]:
# Unit Price > 0.04 
df2 = df2.loc[df2['unit_price'] >= 0.04, :]

# Quantity
df_returns = df2.loc[df2['quantity']<0, :]
df_purchases = df2.loc[df2['quantity']>0, :]

# 3. Feature Engineering

### 3.0 Dataset

In [95]:
df3 = df2.copy()

# Data Reference
df_ref = df3.drop (['invoice_no', 'stock_code',
                    'quantity', 'invoice_date', 'unit_price',
                    'country'], axis=1).drop_duplicates(ignore_index=True)

### 3.1 Gross Revenue

In [96]:
# Monetary (Gross Revenue)
df_purchases['gross_revenue'] = df_purchases['quantity'] * df_purchases['unit_price']
df_monetary = df_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge (df_ref, df_monetary, on = 'customer_id', how = 'left')

df_ref.isna().sum()

customer_id       0
gross_revenue    27
dtype: int64

### 3.2 Recency - Day from last purchase

In [97]:
# Recency
df_recency = df_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df_purchases['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on = 'customer_id', how = 'left')

df_ref.isna().sum()

customer_id       0
gross_revenue    27
recency_days     27
dtype: int64

### 3.3 Quantity of purchases

In [98]:
df_count = (df_purchases[['customer_id', 'invoice_no']].drop_duplicates()
                                                       .groupby('customer_id')
                                                       .count()
                                                       .reset_index()
                                                       .rename(columns={'invoice_no':'purchases_no'}))

df_ref = pd.merge(df_ref, df_count, on = 'customer_id', how = 'left')

### 3.4 Quantity of diferent products purchased

In [99]:
df_prod = (df_purchases[['customer_id', 'stock_code']].groupby('customer_id')
                                                      .count()
                                                      .reset_index()
                                                      .rename(columns={'stock_code':'products_no'}))

df_ref = pd.merge(df_ref, df_prod, on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id       0
gross_revenue    27
recency_days     27
purchases_no     27
products_no      27
dtype: int64

### 3.5 Quantity of items purchased


In [100]:
df_prod = (df_purchases[['customer_id', 'quantity']].groupby('customer_id')
                                                    .sum()
                                                    .reset_index()
                                                    .rename(columns={'quantity':'items_no'}))

df_ref = pd.merge(df_ref, df_prod, on = 'customer_id', how = 'left')
df_ref.isna().sum()


customer_id       0
gross_revenue    27
recency_days     27
purchases_no     27
products_no      27
items_no         27
dtype: int64

### 3.6 Frequency Purchase

In [101]:
df_aux = (df_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                                     .groupby('customer_id')
                                                                     .agg(max_    = ('invoice_date', 'max'),
                                                                          min_    = ('invoice_date', 'min'),
                                                                          days_   = ('invoice_date', lambda x: ((x.max() - x.min()).days) + 1),
                                                                          buy_    = ('invoice_no', 'count') ) ).reset_index()

df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis = 1)

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id       0
gross_revenue    27
recency_days     27
purchases_no     27
products_no      27
items_no         27
frequency        27
dtype: int64

### 3.7 Returns

In [102]:
df_returns.head()

,invoice_no,stock_code,quantity,invoice_date,unit_price,customer_id,country
154,C536383,35004C,-1,2010-12-01,4.65,15311,United Kingdom
939,C536506,22960,-6,2010-12-01,4.25,17897,United Kingdom
1441,C536543,22632,-1,2010-12-01,2.10,17841,United Kingdom
1442,C536543,22355,-2,2010-12-01,0.85,17841,United Kingdom
1973,C536548,22244,-4,2010-12-01,1.95,12472,Germany


In [103]:
# Return Number
df_returns = df_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity' : 'returns_no'})
df_returns['returns_no'] = df_returns['returns_no'] * -1

df_ref = pd.merge(df_ref, df_returns, how = 'left', on='customer_id')
df_ref.loc[df_ref['returns_no'].isna(), 'returns_no'] = 0

df_ref.isna().sum()

customer_id       0
gross_revenue    27
recency_days     27
purchases_no     27
products_no      27
items_no         27
frequency        27
returns_no        0
dtype: int64

### 3.8 Satisfaction Rate

In [104]:
df_ref['satisfaction_rate'] = 1 - df_ref['returns_no'] / df_ref['items_no']

In [105]:
df_ref.isna().sum()

customer_id           0
gross_revenue        27
recency_days         27
purchases_no         27
products_no          27
items_no             27
frequency            27
returns_no            0
satisfaction_rate    27
dtype: int64

### 3.9 Recurrence

In [106]:
df_purchases['invoice_month'] = df_purchases['invoice_date'].dt.month
df_aux = df_purchases[['customer_id', 'invoice_month']].drop_duplicates().groupby(['customer_id']).count().reset_index()
df_aux = df_aux.rename(columns={'invoice_month':'recorrence'})

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'recorrence']], on = 'customer_id', how = 'left')
df_ref.isna().sum()

                                                                  

customer_id           0
gross_revenue        27
recency_days         27
purchases_no         27
products_no          27
items_no             27
frequency            27
returns_no            0
satisfaction_rate    27
recorrence           27
dtype: int64

# 4. Exploratory Data Analysis

### 4.1 Profile Report

In [107]:
df4 = df_ref.dropna().copy()
df4.isna().sum()

customer_id          0
gross_revenue        0
recency_days         0
purchases_no         0
products_no          0
items_no             0
frequency            0
returns_no           0
satisfaction_rate    0
recorrence           0
dtype: int64

# 5 Data Preparation

#### 4.2.0 Data Preparation

In [108]:
df4.head()

,customer_id,gross_revenue,recency_days,purchases_no,products_no,items_no,frequency,returns_no,satisfaction_rate,recorrence
0,17850,5391.21,372.0,34.0,297.0,1733.0,17.000000,40.0,0.976919,1.0
1,13047,3232.59,56.0,9.0,171.0,1390.0,0.028302,35.0,0.974820,7.0
2,12583,6705.38,2.0,15.0,232.0,5028.0,0.040323,50.0,0.990056,11.0
3,13748,948.25,95.0,5.0,28.0,439.0,0.017921,0.0,1.000000,3.0
4,15100,876.00,333.0,3.0,3.0,80.0,0.073171,22.0,0.725000,2.0


In [109]:
# Features Selection
df43 = df4.drop(columns = ['customer_id', 'recency_days', 'purchases_no', 'returns_no', 'satisfaction_rate'], axis=1).copy()

In [110]:
mm = pp.MinMaxScaler()

gross_revenue_scaler = pickle.load(open('features/gross_revenue_scaler.pkl', 'rb'))
df43['gross_revenue'] = mm.fit_transform(df43[['gross_revenue']])

products_no_scaler = pickle.load(open('features/products_no_scaler.pkl', 'rb'))
df43['products_no'] = mm.fit_transform(df43[['products_no']])

items_no_scaler = pickle.load(open('features/items_no_scaler.pkl', 'rb'))
df43['items_no'] = mm.fit_transform(df43[['items_no']])

frequency_scaler = pickle.load(open('features/frequency_scaler.pkl', 'rb'))
df43['frequency'] = mm.fit_transform(df43[['frequency']])

recorrence_scaler = pickle.load(open('features/recorrence_scaler.pkl', 'rb'))
df43['recorrence'] = mm.fit_transform(df43[['recorrence']])


In [111]:
X = df43.copy()
X.head()

,gross_revenue,products_no,items_no,frequency,recorrence
0,0.019301,0.037770,0.008799,1.000000,0.000000
1,0.011567,0.021692,0.007056,0.001345,0.545455
2,0.024009,0.029476,0.025538,0.002052,0.909091
3,0.003384,0.003445,0.002225,0.000734,0.181818
4,0.003125,0.000255,0.000401,0.003985,0.090909


# 6. Machine Learning Modelling

### 6.1 K-Means

In [30]:
# model definition
k=6
kmeans = c.KMeans(init='random',
                  n_clusters=k,
                  n_init=300,
                  max_iter=300,
                  random_state=42)

# model training
kmeans.fit(X)

# clustering
labels = kmeans.labels_

### 6.2 Cluster Validation

In [31]:
# SS
print('SS Value: {}'.format(m.silhouette_score(X, labels, metric='euclidean')))

SS Value: 0.749401061479475


# 7. Evaluation - Cluster Analysis

### 7.1 Visualization Inspection

In [32]:
df9 = X.copy()
df9['cluster'] = labels
df9.head()



,gross_revenue,products_no,items_no,frequency,recorrence,cluster
0,0.019301,0.037770,0.008799,1.000000,0.000000,5
1,0.011567,0.021692,0.007056,0.001345,0.545455,0
2,0.024009,0.029476,0.025538,0.002052,0.909091,2
3,0.003384,0.003445,0.002225,0.000734,0.181818,3
4,0.003125,0.000255,0.000401,0.003985,0.090909,4


### 7.2 Cluster Profile

In [123]:
df92 = df4.copy().reset_index(drop=True)

df92['cluster'] = labels

# Change dtypes
df92['recency_days']    = df92['recency_days'].astype(int)
df92['purchases_no']    = df92['purchases_no'].astype(int)
df92['products_no']     = df92['products_no'].astype(int)
df92['items_no']        = df92['items_no'].astype(int)

In [124]:
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = (df_cluster['customer_id'] / df_cluster['customer_id'].sum())*100

# Avg gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on = 'cluster')

# Avg purchases_no
df_avg_purchases_no = df92[['purchases_no', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_purchases_no, how='inner', on='cluster')

# Avg products_no
df_avg_products_no = df92[['products_no', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_products_no, how='inner', on='cluster')

# Avg items_no
df_avg_items_no = df92[['items_no', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_items_no, how='inner', on='cluster')

# Avg frequency
df_avg_frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_frequency, how='inner', on='cluster')

# Avg recorrence
df_recorrence = df92[['recorrence', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_recorrence, how='inner', on='cluster')


In [125]:
df_cluster.sort_values('gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,purchases_no,products_no,items_no,frequency,recorrence
2,2,162,3.750868,19509.810864,28.882716,558.006173,10972.228395,0.081103,11.018519
0,0,263,6.089373,4862.485779,11.517110,238.045627,3046.779468,0.037414,7.885932
1,1,430,9.956008,2836.961326,6.934884,164.525581,1634.597674,0.027310,5.383721
3,3,915,21.185460,1525.778459,4.109290,93.321311,936.793443,0.025535,3.424044
4,4,918,21.254920,833.931373,2.298475,51.145969,499.606754,0.037875,2.000000
5,5,1631,37.763371,425.130681,1.126303,24.630901,245.707541,1.009846,1.000613


In [126]:
df92.head()

,customer_id,gross_revenue,recency_days,purchases_no,products_no,items_no,frequency,returns_no,satisfaction_rate,recorrence,cluster
0,17850,5391.21,372,34,297,1733,17.000000,40.0,0.976919,1.0,5
1,13047,3232.59,56,9,171,1390,0.028302,35.0,0.974820,7.0,0
2,12583,6705.38,2,15,232,5028,0.040323,50.0,0.990056,11.0,2
3,13748,948.25,95,5,28,439,0.017921,0.0,1.000000,3.0,3
4,15100,876.00,333,3,3,80,0.073171,22.0,0.725000,2.0,4


In [127]:
# Cluster Name

df92['cluster_name'] = 'NA'

for i in range(len(df92)):
    if (df92.loc[i, 'cluster'] == 2):
        df92.loc[i, 'cluster_name'] = 'teste'
    elif (df92.loc[i, 'cluster'] == 0):
        df92.loc[i, 'cluster_name'] = 'pre_insider1'
    elif (df92.loc[i, 'cluster'] == 1):
        df92.loc[i, 'cluster_name'] = 'pre_insider2'
    elif (df92.loc[i, 'cluster'] == 3):
        df92.loc[i, 'cluster_name'] = 'sporadic1'
    elif (df92.loc[i, 'cluster'] == 4):
        df92.loc[i, 'cluster_name'] = 'sporadic2'
    else:
        df92.loc[i, 'cluster_name'] = 'sporadic3' 

# 8. Deploy

In [129]:
df92.dtypes

customer_id            int64
gross_revenue        float64
recency_days           int64
purchases_no           int64
products_no            int64
items_no               int64
frequency            float64
returns_no           float64
satisfaction_rate    float64
recorrence           float64
cluster                int32
cluster_name          object
dtype: object

In [133]:
# Database Conection
#endpoint_local = 'sqlite:////home/vitor/Repos/loyalty-program//data/insiders_db.sqlite'

host = 'database-insiders.cohf4nijv2pv.sa-east-1.rds.amazonaws.com'
port = '5432'
database = 'postgres'
user = 'postgres'
pwd = 'comunidadeds!'

endpoint_aws = f'postgresql://{user}:{pwd}@{host}:{port}/{database}'

engine = create_engine(endpoint_aws)
connection = engine.connect()

# # 1. Drop Table
# query_drop_insiders = """ 
#     DROP TABLE insiders
# """

## 2. Create Table
# query_create_table_insiders = """ 
#     CREATE TABLE insiders(
#         customer_id         INTEGER,
#         gross_revenue       REAL,
#         recency_days        INTEGER,
#         purchases_no        INTEGER,
#         products_no         INTEGER,
#         items_no            INTEGER,
#         frequency           REAL,
#         returns_no          INTEGER,
#         satisfaction_rate   REAL,
#         recorrence          REAL,
#         cluster             INTEGER,
#         cluster_name        VARCHAR(15)
#     )
# """

# engine.execute(query_create_table_insiders)

# 3. Insert Data
df92.to_sql('insiders', con=engine, if_exists='append', index=False)

319

In [134]:
# Consulting database
query = """
    SELECT * FROM insiders
"""

df = pd.read_sql_query(query, engine)

df.head()

,customer_id,gross_revenue,recency_days,purchases_no,products_no,items_no,frequency,returns_no,satisfaction_rate,recorrence,cluster,cluster_name
0,17850,5391.21,372,34,297,1733,17.000000,40,0.976919,1.0,5,sporadic3
1,13047,3232.59,56,9,171,1390,0.028302,35,0.974820,7.0,0,pre_insider1
2,12583,6705.38,2,15,232,5028,0.040323,50,0.990056,11.0,2,teste
3,13748,948.25,95,5,28,439,0.017921,0,1.000000,3.0,3,sporadic1
4,15100,876.00,333,3,3,80,0.073171,22,0.725000,2.0,4,sporadic2


In [136]:
connection.close()